In [ ]:
import numpy as np
import pandas as pd
import jsonlines
from tqdm import tqdm
from joblib import Parallel, delayed
import category_encoders as ce
from pandarallel import pandarallel

pd.set_option('display.max_columns', None)
pandarallel.initialize()

In [ ]:
def reduce_mem_usage(df, verbose=False):
    """
    Utility function to reduce the memory usage of pandas dataframes
    
    Parameters
    ----------
    df: pandas.Dataframe
    verbose: Boolean
    """
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: 
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

***
## preparing train data

In [ ]:
train = pd.read_parquet("../data/train_data.parquet")
train["date"] = pd.to_datetime(train["date"])
train["weekday"] = (train.date.dt.weekday+1).astype(str)
train

In [ ]:
train.groupby("sku")["sold_quantity"].mean()

In [ ]:
train["tmp"] = train.eval("sold_quantity*minutes_active")
q_mean = (train.groupby("sku")["tmp"].sum() / train.groupby("sku")["minutes_active"].sum()).reset_index(name="q_mean")
train = train.merge(q_mean, how="inner", on="sku")
train.drop("tmp", axis=1, inplace=True)

train["tmp"]  = train.eval("minutes_active * (sold_quantity - q_mean)**2")
q_std = (train.groupby("sku")["tmp"].sum() / train.groupby("sku")["minutes_active"].sum()).reset_index(name="q_std")
train = train.merge(q_std, how="inner", on="sku")
train.drop("tmp", axis=1, inplace=True)

In [ ]:
train.loc[:,["currency","listing_type","shipping_logistic_type","shipping_payment"]].nunique()

In [ ]:
encoder = ce.OneHotEncoder(cols=["weekday","listing_type","shipping_logistic_type","shipping_payment"], use_cat_names=True)
train = encoder.fit_transform(train)
train

In [ ]:
test = pd.read_csv("../data/test_data.csv")
train = pd.merge(train, test, how="inner", on="sku")
train

In [ ]:
cols_sum = [
    "listing_type_classic",
    "listing_type_premium",
    "shipping_logistic_type_fulfillment",
    "shipping_logistic_type_cross_docking",
    "shipping_logistic_type_drop_off",
    "shipping_payment_free_shipping",
    "shipping_payment_paid_shipping",
    "weekday_1",
    "weekday_2",
    "weekday_3",
    "weekday_4",
    "weekday_5",
    "weekday_6",
    "weekday_7",    
]
cols_mean = [
    "minutes_active",
]

def compute_features(df):
    target_stock = df.target_stock.values[0]
    start_index = df.index.min()
    end_index = df.index.max()

    if len(df) < 30:
        return None

    inventory_days = list()
    sum_values = list()
    mean_values = list()

    for i in df.index:
        if i+29 > end_index: break

        df_slice = df.loc[i:i+29]
        cumsum = df_slice.sold_quantity.values.cumsum()
        idxs = np.argwhere(cumsum >= target_stock)
        if len(idxs)==0:
            idays = np.inf
        else:
            idays = np.min(idxs) + 1

        inventory_days.append(idays)
        sum_values.append(df_slice[cols_sum].sum(axis=0).values)
        mean_values.append(df_slice[cols_mean].mean(axis=0).values)

    df = df.head(len(inventory_days)).copy()
    df["inventory_days"] = inventory_days
    df[cols_sum] = np.asarray(sum_values)
    df[cols_mean] = np.asarray(mean_values)
    
    return df

with Parallel(n_jobs=6) as parallel:
    delayed_func = delayed(compute_features)
    all_dfs = parallel(delayed_func(df) for _,df in tqdm(train.groupby("sku")))

In [ ]:
train = pd.concat(filter(lambda x: x is not None, all_dfs), ignore_index=True)
train

In [ ]:
# todo: impute zero q_mean & q_std with stats from item_domain_id

 23%|██▎       | 125784/551472 [44:41<2:42:12, 43.74it/s]

In [ ]:
all_records = list()
with jsonlines.open('../data/items_static_metadata_full.jl') as reader:
    for obj in tqdm(reader):
        all_records.append(obj)
        
metadata = pd.DataFrame(all_records)
metadata.drop(["item_title"], axis=1, inplace=True)
metadata

In [ ]:
train = pd.merge(train, metadata, how="inner", on="sku")
train

In [ ]:
train = reduce_mem_usage(train, verbose=True)
train.to_parquet("../data/train-m2.parquet", index=False)

***
## preparing test data

In [ ]:
cols = ["sku","current_price", "currency", "listing_type", "shipping_logistic_type", 
        "shipping_payment", "minutes_active", "available", "item_domain_id", 
        "item_id", "site_id", "product_id", "product_family_id"]

def get_train_feats(df, look_back=7):
    row = dict()
    for col in cols:
        counts = df.tail(look_back).loc[:,col].value_counts()
        if len(counts) == 0:
            row[col] = None 
        else:
            row[col] = counts.index[0]  
    return row

In [ ]:
all_rows = list()
    
with Parallel(n_jobs=6) as parallel:
    delayed_func = delayed(get_train_feats)
    all_rows = parallel(delayed_func(df) for _,df in tqdm(train.groupby("sku")))

train_feats = pd.DataFrame(all_rows)
train_feats

In [ ]:
test = pd.read_csv("../data/test_data.csv")
test = pd.merge(test, train_feats, how="left", on="sku")
test

In [ ]:
all_dfs = list()

for date in pd.date_range("2021-04-01", "2021-04-30"):
    _test = test.copy(deep=True)
    _test["date"] = date
    all_dfs.append(_test)
    
test = pd.concat(all_dfs, ignore_index=True)
test

In [ ]:
test = reduce_mem_usage(test, verbose=True)
test.to_parquet("../data/test-m2.parquet", index=False)

***